In [ ]:
# define path to raw data
raw_data_path = 'raw_data'
all_files = glob.glob(os.path.join(raw_data_path, "*.csv"))

print(f"Found {len(all_files)} CSV files. Starting merge process...")

df_list = []

for filename in all_files:
    try:
        df = pd.read_csv(filename, low_memory=False)
        df_list.append(df)
        print(f"Loaded: {os.path.basename(filename)}")
    except Exception as e:
        print(f"Error loading {filename}: {e}")

# merge all files
if df_list:
    df_raw = pd.concat(df_list, ignore_index=True)
    print(f"\nMerge Complete! Total raw rows: {len(df_raw)}")
else:
    print("No CSV files found. Please check your file path.")

Found 12 CSV files. Starting merge process...
Loaded: 409JourneyDataExtract01Dec2024-14Dec2024.csv
Loaded: 399JourneyDataExtract01Jul2024-14Jul2024.csv
Loaded: 408JourneyDataExtract15Nov2024-30Nov2024.csv
Loaded: 400JourneyDataExtract15Jul2024-31Jul2024.csv
Loaded: 403JourneyDataExtract27Aug2024-17Sep2024.csv
Loaded: 404JourneyDataExtract18Sep2024-30Sep2024.csv
Loaded: 401JourneyDataExtract01Aug2024-14Aug2024.csv
Loaded: 406JourneyDataExtract15Oct2024-31Oct2024.csv
Loaded: 410JourneyDataExtract15Dec2024-31Dec2024.csv
Loaded: 405JourneyDataExtract01Oct2024-14Oct2024.csv
Loaded: 402JourneyDataExtract15Aug2024-26Aug2024.csv
Loaded: 407JourneyDataExtract01Nov2024-14Nov2024.csv

Merge Complete! Total raw rows: 4612029


In [ ]:
df_raw.columns = df_raw.columns.str.strip()

# rename
rename_mapping = {
    'Number': 'Rental_Id',
    'Start date': 'Start_Date',
    'Start station number': 'Start_Station_Id',
    'Start station': 'Start_Station_Name',
    'End date': 'End_Date',
    'End station number': 'End_Station_Id',
    'End station': 'End_Station_Name',
    'Total duration (ms)': 'Duration_ms'
}

# rename columns
df_raw.rename(columns=rename_mapping, inplace=True)

# convert duration 
df_raw['Duration_Secs'] = df_raw['Duration_ms'] / 1000

print("Columns renamed and duration converted.")
display(df_raw[['Start_Date', 'Duration_ms', 'Duration_Secs']].head())

Columns renamed and duration converted.


,Start_Date,Duration_ms,Duration_Secs
0,2024-12-14 23:59,29203,29.203
1,2024-12-14 23:59,1620164,1620.164
2,2024-12-14 23:59,1096981,1096.981
3,2024-12-14 23:59,783763,783.763
4,2024-12-14 23:59,322155,322.155


In [ ]:
# check missing values
print("Missing values before cleaning:")
print(df_raw.isnull().sum())

# create a clean copy
df_clean = df_raw.copy()

# drop rows where Station IDs are missing
df_clean.dropna(subset=['Start_Station_Id', 'End_Station_Id'], inplace=True)

# calculate removed rows
rows_removed = len(df_raw) - len(df_clean)
print(f"\nRows removed due to missing Station IDs: {rows_removed}")


Missing values before cleaning:
Rental_Id             0
Start_Date            0
Start_Station_Id      0
Start_Station_Name    0
End_Date              0
End_Station_Id        0
End_Station_Name      0
Bike number           1
Bike model            0
Total duration        0
Duration_ms           0
Duration_Secs         0
dtype: int64

Rows removed due to missing Station IDs: 0


In [ ]:
# covert time formate
print("Converting timestamps with mixed formats... (This might take a moment)")

df_clean['Start_Date'] = pd.to_datetime(df_clean['Start_Date'], dayfirst=True, format='mixed')
df_clean['End_Date'] = pd.to_datetime(df_clean['End_Date'], dayfirst=True, format='mixed')

# feature engineering
df_clean['Month'] = df_clean['Start_Date'].dt.month
df_clean['Hour'] = df_clean['Start_Date'].dt.hour
df_clean['Day_of_Week'] = df_clean['Start_Date'].dt.dayofweek # 0=Mon, 6=Sun
df_clean['Year'] = df_clean['Start_Date'].dt.year

# mark month
def get_season(month):
    if month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter' # December

df_clean['Season'] = df_clean['Month'].apply(get_season)

# mark weekends
df_clean['Is_Weekend'] = df_clean['Day_of_Week'].apply(lambda x: 1 if x >= 5 else 0)

print("Feature engineering complete.")
display(df_clean[['Start_Date', 'Season', 'Hour', 'Is_Weekend']].head())

Converting timestamps with mixed formats... (This might take a moment)
Feature engineering complete.


,Start_Date,Season,Hour,Is_Weekend
0,2024-12-14 23:59:00,Winter,23,1
1,2024-12-14 23:59:00,Winter,23,1
2,2024-12-14 23:59:00,Winter,23,1
3,2024-12-14 23:59:00,Winter,23,1
4,2024-12-14 23:59:00,Winter,23,1


In [ ]:
min_duration = 60      # 1 minute
max_duration = 10800   # 3 hours

# filter data 
df_final = df_clean[
    (df_clean['Duration_Secs'] >= min_duration) & 
    (df_clean['Duration_Secs'] <= max_duration)
]

outliers_count = len(df_clean) - len(df_final)
percentage = (outliers_count / len(df_clean)) * 100

print(f"Outliers removed: {outliers_count} ({percentage:.2f}%)")
print(f"Final dataset size: {len(df_final)}")

Outliers removed: 52442 (1.14%)
Final dataset size: 4559587


In [ ]:
output_folder = 'cleaned_data'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

output_file = os.path.join(output_folder, 'TfL_2024_Cleaned_Data.csv')

df_final.to_csv(output_file, index=False)

print(f"Success! Cleaned data saved to: {output_file}")

Success! Cleaned data saved to: cleaned_data/TfL_2024_Cleaned_Data.csv
